In [102]:
##import packages

import requests
import json
import pandas as pd
import numpy as np
from decimal import Decimal
import itertools as it

In [103]:
##Request fixtures from FPL API

url = f"https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
json_data = response.json()

Fixtures_Dim = json_data

Fixtures_Dim = pd.json_normalize(Fixtures_Dim)


Fixtures_Dim_Future=Fixtures_Dim[Fixtures_Dim['finished']==False]

### Used as variable to return only the data for completed gameweeks

UpcomingWeek = Fixtures_Dim_Future['event'].min().astype(int)
PrevWeek = UpcomingWeek-1

### Variable for number of gameweeks to go back to calculate averages from

NoWeeks = 4

TeamID = 1769272

print(UpcomingWeek)
print(PrevWeek)


17
16


In [104]:
###Request Team Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

teams_dim = json_data['teams']

Teams_Dim = pd.json_normalize(teams_dim)



In [105]:
###Duplicates of Fixtures for Home and Away, so that this can be merged with elements

Fixtures_Away1 = Fixtures_Dim_Future
Fixtures_Away1.head()

Fixtures_Home1 = Fixtures_Dim_Future
Fixtures_Home1.head()

FixturesConcat = pd.concat([Fixtures_Away1,Fixtures_Home1], keys=["Away", "Home"])

FixturesConcat.reset_index(inplace=True)

FixturesConcat['TeamID'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Away' else row['team_h'], 
    axis=1
)

FixturesConcat['OppTeam2'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Home' else row['team_h'], 
    axis=1
)

In [106]:
### Gameweek elements data from FPL API

player_data = []

for gw1 in range(1,UpcomingWeek):
    url = f"https://fantasy.premierleague.com/api/event/{gw1}/live/"
    
    response = requests.get(url)
    json_data = response.json()
    print(url)

    for gw in json_data['elements']:
        #i1 = gw['id']
        playerid=gw['id']
        minutes=gw['stats']['minutes']
        goals_scored=gw['stats']['goals_scored']
        assists=gw['stats']['assists']
        clean_sheets=gw['stats']['clean_sheets']
        goals_conceded=gw['stats']['goals_conceded']
        own_goals=gw['stats']['own_goals']
        penalties_saved=gw['stats']['penalties_saved']
        penalties_missed=gw['stats']['penalties_missed']
        yellow_cards=gw['stats']['yellow_cards']
        red_cards=gw['stats']['red_cards']
        saves=gw['stats']['saves']
        bonus=gw['stats']['bonus']
        bps=gw['stats']['bps']
        influence=gw['stats']['influence']
        creativity=gw['stats']['creativity']
        threat=gw['stats']['threat']
        ict_index=gw['stats']['ict_index']
        starts=gw['stats']['starts']
        expected_goals=gw['stats']['expected_goals']
        expected_assists=gw['stats']['expected_assists']
        expected_goal_involvements=gw['stats']['expected_goal_involvements']
        expected_goals_conceded=gw['stats']['expected_goals_conceded']
        total_points=gw['stats']['total_points']
        if gw['explain']: 
            fixtureid = gw['explain'][0]['fixture'] 
        else: 
            fixtureid = None

        player_data.append({
            'playerid': playerid,
            'minutes': minutes,
            'goals_scored': goals_scored,
            'assists': assists,
            'clean_sheets': clean_sheets,
            'goals_conceded': goals_conceded,
            'threat': threat,
            'expected_goals': expected_goals,
            'expected_assists': expected_assists,
            'expected_goal_involvements': expected_goal_involvements,
            'expected_goals_conceded': expected_goals_conceded,
            'fixtureid': fixtureid  


            })

    GWElementsDF = pd.DataFrame(player_data)
GWElementsDF.head(5)

GWElementsDF1 = pd.merge(GWElementsDF, Fixtures_Dim,left_on='fixtureid', right_on='id', how='outer')

GWElementsDF1.to_csv('FPL_Event_By_Week.csv')
GWElementsDF1.head()


https://fantasy.premierleague.com/api/event/1/live/
https://fantasy.premierleague.com/api/event/2/live/
https://fantasy.premierleague.com/api/event/3/live/
https://fantasy.premierleague.com/api/event/4/live/
https://fantasy.premierleague.com/api/event/5/live/
https://fantasy.premierleague.com/api/event/6/live/
https://fantasy.premierleague.com/api/event/7/live/
https://fantasy.premierleague.com/api/event/8/live/
https://fantasy.premierleague.com/api/event/9/live/
https://fantasy.premierleague.com/api/event/10/live/
https://fantasy.premierleague.com/api/event/11/live/
https://fantasy.premierleague.com/api/event/12/live/
https://fantasy.premierleague.com/api/event/13/live/
https://fantasy.premierleague.com/api/event/14/live/
https://fantasy.premierleague.com/api/event/15/live/
https://fantasy.premierleague.com/api/event/16/live/


,playerid,minutes_x,goals_scored,assists,clean_sheets,goals_conceded,threat,expected_goals,expected_assists,expected_goal_involvements,...,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
1,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
2,3.0,90.0,0.0,0.0,1.0,0.0,8.0,0.12,0.02,0.14,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
3,4.0,90.0,1.0,1.0,1.0,0.0,46.0,0.45,0.04,0.49,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0


In [107]:
###Standard Element Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

elements_dim = json_data['elements']

Elements_Dim1 = pd.json_normalize(elements_dim)

Elements_Dim1.to_csv('elementsdim.csv')

Elements_Dim1 = pd.merge(GWElementsDF1, Elements_Dim1,left_on='playerid', right_on='id', how='outer',suffixes=('.gw', '.info'))

Elements_Dim1['ExcludeDef'] = Elements_Dim1.apply(
    lambda row: 'Y' if (row['element_type'] == '1' or row['element_type'] == '2') and row['minutes_x'] > 59 else 'N', 
    axis=1
)

Elements_Dim1.to_csv('elements4wk.csv')

In [108]:
###Arbitrary team weightings calculated based on average of last season's odds

TeamWeightings = {
    "Weighting": [
        1.699153895, 1.68275948, 1.639154108, 1.626396476, 1.605876872, 1.542304678,
        1.526424188, 1.50219836, 1.499125082, 1.489893392, 1.44492154, 1.424635514,
        1.365324148, 1.288771283, 1.266010528, 1.251813146, 1.18226904, 1.7, 1.7, 1.7
    ],
    "Teams.id": [
        3, 7, 16, 20, 4, 9, 8, 2, 15, 19, 5, 14, 6, 12, 18, 1, 13, 10, 11, 17
    ],
    "Teams.name": [
        "Bournemouth", "Crystal Palace", "Nott'm Forest", "Wolves", "Brentford", "Fulham",
        "Everton", "Aston Villa", "Newcastle", "West Ham", "Brighton", "Man Utd", "Chelsea",
        "Liverpool", "Spurs", "Arsenal", "Man City", "Ipswich", "Leicester", "Southampton"
    ]
}

TeamWeightings = pd.DataFrame(TeamWeightings)

Elements_Dim1['OppTeam'] = Elements_Dim1.apply(lambda row: row['team_h'] if row['team_h'] == row['team'] else row['team_a'], axis=1)

Elements_Dim2 = pd.merge(Elements_Dim1, TeamWeightings,left_on='OppTeam', right_on='Teams.id', how='outer')

Elements_Dim2['XGC_Weighted'] = Elements_Dim2['Weighting'].astype(float)*Elements_Dim2['expected_goals_conceded.gw'].astype(float)

Elements_Dim2['XGI_Weighted'] = Elements_Dim2['expected_goal_involvements.gw'].astype(float)/Elements_Dim2['Weighting'].astype(float)



In [109]:
###Filtered to only include the previous 4 weeks' data before it is grouped to return total sum for each team 

Elements_Dim2Filtered = Elements_Dim2[Elements_Dim2['event']>PrevWeek-4]

Elements_Dim2Grouped1 = Elements_Dim2Filtered.groupby(['OppTeam', 'event']).agg(
    XGI_Weighted=('XGI_Weighted', 'sum'),
    XGC_Weighted=('XGC_Weighted', 'max'),
    Minutes=('minutes_x', 'sum')
).reset_index()



##Average of the above taken for each team
Elements_Dim2Grouped2 = Elements_Dim2Grouped1.groupby(['OppTeam']).agg(
    XGI_Weighted=('XGI_Weighted', 'mean'),
    XGC_Weighted=('XGC_Weighted', 'mean'),
    Minutes=('Minutes', 'mean')
).reset_index()



Elements_Dim2Grouped2.head()

,OppTeam,XGI_Weighted,XGC_Weighted,Minutes
0,1.0,0.810025,0.629036,263.000000
1,2.0,0.497493,1.783861,263.133333
2,3.0,0.622663,1.928540,262.800000
3,4.0,0.433823,2.890578,262.933333
4,5.0,0.453543,1.910909,262.866667


In [110]:
##Previous 4 Gameweeks this dataframe is returning the averages for the selected players 
Elements_Dim1 = Elements_Dim1[Elements_Dim1['event']<UpcomingWeek]

GWElementsDFLast4Weeks = Elements_Dim1[Elements_Dim1['event']>UpcomingWeek-NoWeeks-1]
GWElementsDFLast4Weeks = Elements_Dim1[['minutes_x','playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']]

GWElementsDFLast4Weeks['expected_goals.gw'] = GWElementsDFLast4Weeks['expected_goals.gw'].astype(float)
GWElementsDFLast4Weeks['expected_assists.gw'] = GWElementsDFLast4Weeks['expected_assists.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goal_involvements.gw'] = GWElementsDFLast4Weeks['expected_goal_involvements.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goals_conceded.gw'] = GWElementsDFLast4Weeks['expected_goals_conceded.gw'].astype(float)

GWElementsDFLast4Weeks = GWElementsDFLast4Weeks.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['minutes_x','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']].mean(numeric_only=True)

GWElementsDFLast4Weeks['ExcludeDef'] = GWElementsDFLast4Weeks.apply(
    lambda row: 'Y' if (row['element_type'] == 1 or row['element_type'] == 2) and row['minutes_x']< 59 else 'N', 
    axis=1
)

grouped_dfLast4Weeks=GWElementsDFLast4Weeks[GWElementsDFLast4Weeks['ExcludeDef']!='Y']
grouped_dfLast4Weeks.head()

C:\Users\james\AppData\Local\Temp\ipykernel_31244\543277916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GWElementsDFLast4Weeks['expected_goals.gw'] = GWElementsDFLast4Weeks['expected_goals.gw'].astype(float)
C:\Users\james\AppData\Local\Temp\ipykernel_31244\543277916.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GWElementsDFLast4Weeks['expected_assists.gw'] = GWElementsDFLast4Weeks['expected_assists.gw'].astype(float)
C:\Users\james\AppData\Local\Temp\ipykernel_31244\543277916.py:9: SettingWit

,playerid,web_name,element_type,now_cost,team,minutes_x,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,ExcludeDef
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.0000,0.000000,0.000000,0.00000,0.000000,N
1,2.0,G.Jesus,4.0,67.0,1.0,16.5000,0.063750,0.008750,0.07250,0.152500,N
2,3.0,Gabriel,2.0,62.0,1.0,73.6250,0.110000,0.033750,0.14375,0.786875,N
3,4.0,Havertz,4.0,79.0,1.0,83.3125,0.363125,0.078125,0.44125,0.871250,N
5,6.0,J.Timber,2.0,57.0,1.0,70.6250,0.041875,0.048125,0.09000,0.633750,N


In [111]:
### Returns combinations of historic XG stats from players multiplied by historic opposition team performance

OpportunitiesCalc = pd.merge(grouped_dfLast4Weeks, Elements_Dim2Grouped2,left_on='team', right_on='OppTeam', how='outer')

OpportunitiesCalc['AttackOpp2'] = OpportunitiesCalc['XGC_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc['DefenceOpp2'] = OpportunitiesCalc['XGI_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc.sort_values(by=['AttackOpp2'])


,playerid,web_name,element_type,now_cost,team,minutes_x,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,ExcludeDef,OppTeam,XGI_Weighted,XGC_Weighted,Minutes,AttackOpp2,DefenceOpp2
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.0000,0.000000,0.000000,0.000000,0.00000,N,1.0,0.810025,0.629036,263.000000,0.000000,0.000000
199,125.0,Gross,3.0,65.0,5.0,0.0000,0.000000,0.000000,0.000000,0.00000,N,5.0,0.453543,1.910909,262.866667,0.000000,0.000000
201,127.0,I.Osman,3.0,50.0,5.0,0.0000,0.000000,0.000000,0.000000,0.00000,N,5.0,0.453543,1.910909,262.866667,0.000000,0.000000
203,130.0,Kozlowski,3.0,45.0,5.0,0.0000,0.000000,0.000000,0.000000,0.00000,N,5.0,0.453543,1.910909,262.866667,0.000000,0.000000
204,132.0,March,3.0,65.0,5.0,0.0000,0.000000,0.000000,0.000000,0.00000,N,5.0,0.453543,1.910909,262.866667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,514.0,Bowen,3.0,74.0,19.0,88.5625,0.206250,0.254375,0.460625,1.77875,N,19.0,0.594226,3.281490,263.000000,1.511536,0.273715
333,328.0,M.Salah,3.0,134.0,12.0,87.8000,0.698667,0.219333,0.918000,0.97200,N,12.0,0.613541,1.774208,206.214286,1.628723,0.563231
241,306.0,Vardy,4.0,56.0,11.0,74.4375,0.293750,0.094375,0.388125,1.90375,N,11.0,0.289804,4.602750,262.866667,1.786442,0.112480
351,351.0,Haaland,4.0,149.0,13.0,89.9375,0.811875,0.054375,0.866250,1.47750,N,13.0,0.562196,2.308380,262.866667,1.999634,0.487002


In [112]:
##### Returns Output 1 FixturesElementsAll
FixturesElementsAll1 = pd.merge(FixturesConcat, Elements_Dim2Grouped2,left_on='OppTeam2', right_on='OppTeam', how='outer')

FixturesElementsAll2 = pd.merge(grouped_dfLast4Weeks, FixturesElementsAll1,left_on='team', right_on='TeamID', how='outer')

FixturesElementsAll2['AttackOpp_pergame'] = FixturesElementsAll2['XGC_Weighted']*FixturesElementsAll2['expected_goal_involvements.gw']

FixturesElementsAll2['DefenceOpp_pergame'] = FixturesElementsAll2['XGI_Weighted']*FixturesElementsAll2['expected_goals_conceded.gw']

FixturesElementsAll2.to_csv('FixturesElementsAll.csv')

In [113]:
##### Returns Output 2 FixturesElementsAllGrouped

FixturesElementsAllGrouped = FixturesElementsAll2[FixturesElementsAll2['event']>UpcomingWeek-NoWeeks-1]
FixturesElementsAllGrouped = FixturesElementsAllGrouped[['playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']]

FixturesElementsAllGrouped = FixturesElementsAllGrouped.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']].mean(numeric_only=True)

FixturesElementsAllGrouped.to_csv('FixturesElementsAllGrouped.csv')

In [114]:
url = f"https://fantasy.premierleague.com/api/entry/{TeamID}/event/{PrevWeek}/picks/"
response = requests.get(url)
json_data = response.json()

TeamSelection = json_data['picks']

TeamSelection = pd.json_normalize(TeamSelection)

TeamSelection.head()

,element,position,multiplier,is_captain,is_vice_captain,element_type
0,15,1,1,False,False,1
1,338,2,1,False,False,2
2,573,3,1,False,False,2
3,422,4,1,False,False,2
4,17,5,1,False,False,3


In [115]:
FixturesElementsAllGroupedwithTS = pd.merge(FixturesElementsAllGrouped, TeamSelection,left_on='playerid', right_on='element', how='outer')

FixturesElementsAllGroupedwithTS['IsInCurrentTeam'] = np.where(
    FixturesElementsAllGroupedwithTS['is_captain'].notnull(),
    'Y',
    'N'
)

In [116]:
FixturesElementsAllGroupedwithTS.head()

,playerid,web_name,element_type_x,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,XGI_Weighted,XGC_Weighted,AttackOpp_pergame,DefenceOpp_pergame,element,position,multiplier,is_captain,is_vice_captain,element_type_y,IsInCurrentTeam
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.000000,0.000000,0.00000,0.000000,0.438089,2.54751,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,N
1,2.0,G.Jesus,4.0,67.0,1.0,0.063750,0.008750,0.07250,0.152500,0.438089,2.54751,0.184694,0.066809,NaN,NaN,NaN,NaN,NaN,NaN,N
2,3.0,Gabriel,2.0,62.0,1.0,0.110000,0.033750,0.14375,0.786875,0.438089,2.54751,0.366205,0.344721,NaN,NaN,NaN,NaN,NaN,NaN,N
3,4.0,Havertz,4.0,79.0,1.0,0.363125,0.078125,0.44125,0.871250,0.438089,2.54751,1.124089,0.381685,NaN,NaN,NaN,NaN,NaN,NaN,N
4,6.0,J.Timber,2.0,57.0,1.0,0.041875,0.048125,0.09000,0.633750,0.438089,2.54751,0.229276,0.277639,NaN,NaN,NaN,NaN,NaN,NaN,N


In [117]:
FixturesElementsAllGroupedwithTS.to_csv('FixturesElementsAllGroupedwithTS.csv')

In [118]:
##Import the packages to carry out the optimisation problem

import requests
from sklearn.preprocessing import MinMaxScaler
import warnings
from pulp import *
Small_Elements_df = FixturesElementsAllGroupedwithTS

In [119]:
def AttackOptimisationproblem(Source,AttSpend,FreeTransfers):
    Attack_Elements_df = Source.dropna(subset=['AttackOpp_pergame'])
    Attack_Elements_df['AttackOpp_pergame'].astype(float)
    Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type_x']!=1]
    Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type_x']!=2]
    ## Attack Variables
    Players = list(Attack_Elements_df['web_name'])
    Cost = dict(zip(Players, Attack_Elements_df['now_cost']))
    Score = dict(zip(Players, Attack_Elements_df['AttackOpp_pergame'].astype(float)))
    positions = dict(zip(Players, Attack_Elements_df['element_type_x']))
    CurrentSelection = dict(zip(Players, Attack_Elements_df['IsInCurrentTeam']))
    
    player_vars = LpVariable.dicts("Player", Players, lowBound=0, upBound=1, cat='Integer')

    ##Attack Constraints and problem solve

    total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)
    total_score += lpSum([Score[i] * player_vars[i] for i in player_vars])
    total_score += lpSum([Cost[i] * player_vars[i] for i in player_vars]) <= AttSpend

    MF = [p for p in positions.keys() if positions[p] == 3]
    ST = [p for p in positions.keys() if positions[p] == 4]
    CS = [p for p in positions.keys() if CurrentSelection[p] == 'N']
    total_score += lpSum([player_vars[i] for i in MF]) == 5
    total_score += lpSum([player_vars[i] for i in ST]) == 3
    total_score += lpSum([player_vars[i] for i in CS]) == 8-FreeTransfers

    total_score.solve()

    for v in total_score.variables():
        if v.varValue > 0:
            print(v.name)


In [120]:
def DefenceOptimisationproblem(Source,DefSpend,FreeTransfers):
    Def_Elements_df=Source.dropna(subset=['DefenceOpp_pergame'])
    Def_Elements_df['DefenceOpp_pergame'].astype(float)
    Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=3]
    Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=4]
    ## Defence Variables
    DefPlayers = list(Def_Elements_df['web_name'])
    DefCost = dict(zip(DefPlayers, Def_Elements_df['now_cost']))
    DefScore = dict(zip(DefPlayers, Def_Elements_df['DefenceOpp_pergame'].astype(float)))
    Defpositions = dict(zip(DefPlayers, Def_Elements_df['element_type_x']))
    DefCurrentSelection = dict(zip(DefPlayers, Def_Elements_df['IsInCurrentTeam']))

    player_vars_def = LpVariable.dicts("DefPlayers", DefPlayers, lowBound=0, upBound=1, cat='Integer')

##Defence Constraints and problem solve

    total_score_def = LpProblem("Fantasy_Points_Problem_Defence", LpMinimize)
    total_score_def += lpSum([DefScore[i] * player_vars_def[i] for i in player_vars_def])
    total_score_def += lpSum([DefCost[i] * player_vars_def[i] for i in player_vars_def]) <= DefSpend

    GK = [p for p in Defpositions.keys() if Defpositions[p] == 1]
    DF = [p for p in Defpositions.keys() if Defpositions[p] == 2]
    #CS = [p for p in DefCurrentSelection.keys() if DefCurrentSelection[p] == 'N']
    total_score_def += lpSum([player_vars_def[i] for i in GK]) == 2
    total_score_def += lpSum([player_vars_def[i] for i in DF]) == 5
    #total_score_def += lpSum([player_vars_def[i] for i in CS]) == 7-FreeTransfers

    total_score_def.solve()

    for v in total_score_def.variables():
        if v.varValue > 0:
            print(v.name)

In [121]:
AttackOptimisationproblem(Small_Elements_df,650,1)
DefenceOptimisationproblem(Small_Elements_df,350,1)

Player_Andreas
Player_B.Fernandes
Player_Evanilson
Player_Johnson
Player_M.Salah
Player_N.Jackson
Player_Palmer
Player_Raúl
DefPlayers_Arrizabalaga
DefPlayers_J.Timber
DefPlayers_Konaté
DefPlayers_Raya
DefPlayers_Romero
DefPlayers_Smith
DefPlayers_W.Fofana
